# Classification with stumbleupon data

Project 4 has been changed since scraping was untenable. The project now focuses on the stumbleupon kaggle dataset. For more information on this dataset, [check out the website here](https://www.kaggle.com/c/stumbleupon).

---

## 1. Load in the dataset

This is the only part completed for you.

---

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('white')

%matplotlib inline

In [49]:
su = pd.read_csv('../dataset/evergreen.tsv', delimiter='\t')

## 2. Clean up/examine your data

Some of the columns may have values that need changing or that are of the wrong type. There could also be columns that aren't very useful.

---

In [50]:
su.dtypes

url                                object
urlid                               int64
boilerplate                        object
alchemy_category                   object
alchemy_category_score             object
avglinksize                       float64
commonlinkratio_1                 float64
commonlinkratio_2                 float64
commonlinkratio_3                 float64
commonlinkratio_4                 float64
compression_ratio                 float64
embed_ratio                       float64
framebased                          int64
frameTagRatio                     float64
hasDomainLink                       int64
html_ratio                        float64
image_ratio                       float64
is_news                            object
lengthyLinkDomain                   int64
linkwordscore                       int64
news_front_page                    object
non_markup_alphanum_characters      int64
numberOfLinks                       int64
numwords_in_url                   

In [51]:
su.alchemy_category_score

0       0.789131
1       0.574147
2       0.996526
3       0.801248
4       0.719157
5              ?
6        0.22111
7              ?
8              ?
9              ?
10      0.816604
11       0.89156
12      0.872323
13             ?
14      0.559991
15      0.548963
16      0.598149
17       0.77292
18      0.287884
19      0.585389
20      0.941129
21      0.437599
22      0.644872
23             ?
24             ?
25      0.898192
26      0.348995
27          0.85
28             ?
29             ?
          ...   
7365           ?
7366           ?
7367           ?
7368           ?
7369           ?
7370    0.749882
7371    0.627681
7372    0.393665
7373     0.70019
7374    0.882445
7375    0.483953
7376    0.332504
7377    0.949532
7378    0.648793
7379    0.657172
7380           ?
7381     0.22311
7382    0.395289
7383    0.307736
7384           ?
7385           ?
7386    0.832741
7387     0.31693
7388    0.679419
7389           ?
7390    0.651067
7391     0.14192
7392    0.1962

In [52]:
def convert_score_to_float(x):
    try:
        return float(x)
    except:
        return np.nan
    
su['alchemy_category_score'] = su.alchemy_category_score.map(convert_score_to_float)

In [53]:
su.alchemy_category_score[0:10]

0    0.789131
1    0.574147
2    0.996526
3    0.801248
4    0.719157
5         NaN
6    0.221110
7         NaN
8         NaN
9         NaN
Name: alchemy_category_score, dtype: float64

In [54]:
su.is_news.unique()

array(['1', '?'], dtype=object)

In [55]:
su['is_news'] = su.is_news.map(lambda x: 1 if x == '1' else 0)

In [56]:
su.is_news.unique()

array([1, 0])

In [57]:
su.is_news.value_counts()

1    4552
0    2843
Name: is_news, dtype: int64

In [58]:
su.news_front_page.unique()

array(['0', '?', '1'], dtype=object)

In [59]:
su.news_front_page.value_counts()

0    5853
?    1248
1     294
Name: news_front_page, dtype: int64

In [60]:
su.loc[su.is_news == 0, 'news_front_page'] = '0'

In [61]:
su.news_front_page.value_counts()

0    7135
1     260
Name: news_front_page, dtype: int64

In [62]:
su['news_front_page'] = su.news_front_page.map(lambda x: int(x))

In [63]:
su.dtypes

url                                object
urlid                               int64
boilerplate                        object
alchemy_category                   object
alchemy_category_score            float64
avglinksize                       float64
commonlinkratio_1                 float64
commonlinkratio_2                 float64
commonlinkratio_3                 float64
commonlinkratio_4                 float64
compression_ratio                 float64
embed_ratio                       float64
framebased                          int64
frameTagRatio                     float64
hasDomainLink                       int64
html_ratio                        float64
image_ratio                       float64
is_news                             int64
lengthyLinkDomain                   int64
linkwordscore                       int64
news_front_page                     int64
non_markup_alphanum_characters      int64
numberOfLinks                       int64
numwords_in_url                   

In [64]:
su.isnull().sum()

url                                  0
urlid                                0
boilerplate                          0
alchemy_category                     0
alchemy_category_score            2342
avglinksize                          0
commonlinkratio_1                    0
commonlinkratio_2                    0
commonlinkratio_3                    0
commonlinkratio_4                    0
compression_ratio                    0
embed_ratio                          0
framebased                           0
frameTagRatio                        0
hasDomainLink                        0
html_ratio                           0
image_ratio                          0
is_news                              0
lengthyLinkDomain                    0
linkwordscore                        0
news_front_page                      0
non_markup_alphanum_characters       0
numberOfLinks                        0
numwords_in_url                      0
parametrizedLinkRatio                0
spelling_errors_ratio    

In [65]:
su.alchemy_category.value_counts()

?                     2342
recreation            1229
arts_entertainment     941
business               880
health                 506
sports                 380
culture_politics       343
computer_internet      296
science_technology     289
gaming                  76
religion                72
law_crime               31
unknown                  6
weather                  4
Name: alchemy_category, dtype: int64

In [66]:
mask = (su.alchemy_category == '?')

print su.loc[mask, 'alchemy_category_score'].isnull().sum()

2342


In [67]:
su = su[~mask]
print su.shape

(5053, 27)


In [68]:
su = su[~su.alchemy_category.isin(['unknown','weather'])]
print su.shape

(5043, 27)


In [74]:
for c in su.columns:
    print c.upper()
    print su[[c]].describe()
    print '-------------------------------------\n'
    

URL
                                                      url
count                                                5043
unique                                               5043
top     http://www.dailydawdle.com/2010/12/polar-bear-...
freq                                                    1
-------------------------------------

URLID
              urlid
count   5043.000000
mean    5313.740631
std     3063.653950
min        5.000000
25%     2651.500000
50%     5312.000000
75%     7985.500000
max    10566.000000
-------------------------------------

BOILERPLATE
                                              boilerplate
count                                                5043
unique                                               5043
top     {"title":"Dailymotion Food For Thought food fo...
freq                                                    1
-------------------------------------

ALCHEMY_CATEGORY
       alchemy_category
count              5043
unique               11
top         

In [75]:
su.drop(['framebased','embed_ratio','image_ratio'], axis=1, inplace=True)

## 3. Use statsmodels' logistic regression function to look at variable significance

The **`import statsmodels.formula.api as smf`** code below gives us access to a statsmodels api that can run logistic regressions using patsy-style formulas.

Ex:

```python
formula = 'target ~ var1 + var2 + C(var3) -1'
logreg = smf.logit(formula, data=data)
logreg_results = logreg.fit()
print logreg_results.summary()
```

---

In [77]:
import statsmodels.formula.api as smf

### 3.1 Run a logistic regression predicting evergreen from the numeric columns

And print out the results as shown in the example above.

---

In [76]:
su.dtypes

url                                object
urlid                               int64
boilerplate                        object
alchemy_category                   object
alchemy_category_score            float64
avglinksize                       float64
commonlinkratio_1                 float64
commonlinkratio_2                 float64
commonlinkratio_3                 float64
commonlinkratio_4                 float64
compression_ratio                 float64
frameTagRatio                     float64
hasDomainLink                       int64
html_ratio                        float64
is_news                             int64
lengthyLinkDomain                   int64
linkwordscore                       int64
news_front_page                     int64
non_markup_alphanum_characters      int64
numberOfLinks                       int64
numwords_in_url                     int64
parametrizedLinkRatio             float64
spelling_errors_ratio             float64
label                             

In [81]:
new_cols = su.columns[:-1].values.tolist()
new_cols.extend(['evergreen'])
su.columns = new_cols
print su.columns

Index([u'url', u'urlid', u'boilerplate', u'alchemy_category',
       u'alchemy_category_score', u'avglinksize', u'commonlinkratio_1',
       u'commonlinkratio_2', u'commonlinkratio_3', u'commonlinkratio_4',
       u'compression_ratio', u'frameTagRatio', u'hasDomainLink', u'html_ratio',
       u'is_news', u'lengthyLinkDomain', u'linkwordscore', u'news_front_page',
       u'non_markup_alphanum_characters', u'numberOfLinks', u'numwords_in_url',
       u'parametrizedLinkRatio', u'spelling_errors_ratio', u'evergreen'],
      dtype='object')


In [83]:
predictor_cols = [c for c in su.columns if c not in ['urlid','evergreen','boilerplate','alchemy_category','url']]
print predictor_cols

['alchemy_category_score', 'avglinksize', 'commonlinkratio_1', 'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4', 'compression_ratio', 'frameTagRatio', 'hasDomainLink', 'html_ratio', 'is_news', 'lengthyLinkDomain', 'linkwordscore', 'news_front_page', 'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url', 'parametrizedLinkRatio', 'spelling_errors_ratio']


In [90]:
su_n = su.copy()
su_n.loc[:, predictor_cols] = (su_n.loc[:, predictor_cols] - su_n.loc[:, predictor_cols].mean()) / su_n.loc[:, predictor_cols].std()

In [91]:
su_n.head(3)

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,evergreen
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.871317,-0.087032,1.071720,-0.054364,-0.477240,-0.364917,...,0.645361,0.696652,-0.300210,-0.202666,-0.002735,-0.057109,0.899716,-0.103015,-0.299090,0
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,-0.138219,0.072915,0.222833,0.521587,1.309024,1.327566,...,0.645361,0.696652,0.497056,-0.202666,-0.066925,0.052936,1.205730,0.060645,0.467734,1
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,1.845216,-0.054762,0.494933,0.750441,0.305338,-0.097350,...,0.645361,0.696652,1.244494,-0.202666,-0.455909,0.512532,1.817758,-0.025226,-0.655305,1


In [92]:
formula = 'evergreen ~ ' + ' + '.join(predictor_cols)
print formula
model_num = smf.logit(formula, data=su_n)

evergreen ~ alchemy_category_score + avglinksize + commonlinkratio_1 + commonlinkratio_2 + commonlinkratio_3 + commonlinkratio_4 + compression_ratio + frameTagRatio + hasDomainLink + html_ratio + is_news + lengthyLinkDomain + linkwordscore + news_front_page + non_markup_alphanum_characters + numberOfLinks + numwords_in_url + parametrizedLinkRatio + spelling_errors_ratio


In [93]:
num_results = model_num.fit()
print num_results.summary()

Optimization terminated successfully.
         Current function value: 0.644566
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:              evergreen   No. Observations:                 5043
Model:                          Logit   Df Residuals:                     5023
Method:                           MLE   Df Model:                           19
Date:                Fri, 27 May 2016   Pseudo R-squ.:                 0.06910
Time:                        10:37:11   Log-Likelihood:                -3250.5
converged:                       True   LL-Null:                       -3491.8
                                        LLR p-value:                 2.483e-90
                                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------------
Intercept                          0.0685      0.030      2.309     

### 3.2 Run a logistic regression predicting evergreen from the numeric columns and a categorical variable of alchemy_category

And print out the results as shown in the example.

---

In [94]:
formula_cat = formula + ' + C(alchemy_category)'

model_cat = smf.logit(formula_cat, data=su_n)
cat_results = model_cat.fit()

print cat_results.summary()

Optimization terminated successfully.
         Current function value: 0.593814
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:              evergreen   No. Observations:                 5043
Model:                          Logit   Df Residuals:                     5013
Method:                           MLE   Df Model:                           29
Date:                Fri, 27 May 2016   Pseudo R-squ.:                  0.1424
Time:                        10:43:27   Log-Likelihood:                -2994.6
converged:                       True   LL-Null:                       -3491.8
                                        LLR p-value:                1.263e-190
                                                coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------------------------------
Intercept                                    -

## 4. Use sklearn to cross-validate the accuracy of the model above

Normalize the numeric and categorical columns of the predictor matrix.

---

In [95]:
import patsy

Y, X = patsy.dmatrices(formula_cat, data=su_n)
Y = np.ravel(Y)


In [96]:
print Y.shape, X.shape

(5043,) (5043, 30)


In [97]:
from sklearn.linear_model import LogisticRegression

logreg_cat = LogisticRegression()

from sklearn.cross_validation import cross_val_score

logreg_scores = cross_val_score(logreg_cat, X, Y, cv=11)

print logreg_scores
print np.mean(logreg_scores)

[ 0.64270153  0.71023965  0.70588235  0.708061    0.63616558  0.69432314
  0.69432314  0.70087336  0.67248908  0.6768559   0.69432314]
0.685112534714


In [98]:
print 'baseline accuracy:', np.mean(Y)

baseline accuracy: 0.519135435257


In [105]:
logreg_rocauc_scores = cross_val_score(logreg_cat, X, np.array(Y), cv=11, scoring='roc_auc')
print np.mean(logreg_rocauc_scores)

0.745845244721


## 5. Gridsearch regularization parameters for logistic regression

Find the best regularization type (Ridge, Lasso) across a set of regularization strengths.

[NOTE: C is the inverse of the regularization strength. Lower C values are stronger regularization. Having a C higher than 1 will significantly slow down the search. I'm not particularly interested in values over 1, since this is the default regularization strength in LogisticRegression.]

**After you find the best set of parameters, build a Logistic Regression with those parameters and crossvalidate the score.**

[NOTE 2: to run Lasso regularization the solver should be `'liblinear'`]

---

In [117]:
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split


Y = np.array(Y)

Xtr, Xte, Ytr, Yte = train_test_split(X, Y, test_size=.2)

lr = LogisticRegression()

lr_params = {
    'solver':['liblinear'],
    'penalty':['l1','l2'],
    'C':np.logspace(-5,1,20)
}

lr_gs = GridSearchCV(lr, lr_params, cv=11, verbose=1)

lr_gs.fit(Xtr, Ytr)

print lr_gs.best_params_
print lr_gs.best_score_

print 'conservative test score:', lr_gs.best_estimator_.score(Xte, Yte)

[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:    1.4s


Fitting 11 folds for each of 40 candidates, totalling 440 fits
{'penalty': 'l2', 'C': 0.01438449888287663, 'solver': 'liblinear'}
0.691125433813
conservative test score: 0.689791873142


[Parallel(n_jobs=1)]: Done 440 out of 440 | elapsed:    5.9s finished


In [109]:
[round(x*100000)/100000. for x in np.logspace(-4,1,20)]

[0.0001,
 0.00018,
 0.00034,
 0.00062,
 0.00113,
 0.00207,
 0.00379,
 0.00695,
 0.01274,
 0.02336,
 0.04281,
 0.07848,
 0.14384,
 0.26367,
 0.48329,
 0.88587,
 1.62378,
 2.97635,
 5.45559,
 10.0]

In [112]:
[round(x*100000)/100000. for x in np.linspace(0.0001,1,20)]

[0.0001,
 0.05273,
 0.10535,
 0.15798,
 0.21061,
 0.26323,
 0.31586,
 0.36848,
 0.42111,
 0.47374,
 0.52636,
 0.57899,
 0.63162,
 0.68424,
 0.73687,
 0.78949,
 0.84212,
 0.89475,
 0.94737,
 1.0]

## 6. Gridsearch neighbors for kNN

Find the best number of neighbors with your predictors to predict the `label` target variable.

Start by bulding a kNN model with a set number of neighbors, then use gridsearch to run through a series of neighbors.

---

In [118]:
from sklearn.neighbors import KNeighborsClassifier

knn_params = {
    'n_neighbors':range(1,101),
    'weights':['uniform','distance']
}

knn = KNeighborsClassifier()

knn_gs = GridSearchCV(knn, knn_params, cv=11, verbose=1)

knn_gs.fit(X, Y)

knn_gs.best_params_
knn_gs.best_score_


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    5.2s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   25.0s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:   58.1s
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:  1.7min
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:  2.8min
[Parallel(n_jobs=1)]: Done 1799 tasks       | elapsed:  4.1min


Fitting 11 folds for each of 200 candidates, totalling 2200 fits


[Parallel(n_jobs=1)]: Done 2200 out of 2200 | elapsed:  5.0min finished


0.67737457862383499

In [119]:
knn_gs.best_params_

{'n_neighbors': 26, 'weights': 'uniform'}

## 7. Choose a new target from alchemy_category to predict with logistic regression

**Ideally your category choice will have a small fraction of the total rows, but not TOO small!**

---

In [120]:
su_n.alchemy_category.value_counts()

recreation            1229
arts_entertainment     941
business               880
health                 506
sports                 380
culture_politics       343
computer_internet      296
science_technology     289
gaming                  76
religion                72
law_crime               31
Name: alchemy_category, dtype: int64

### 7.1 Chose your target category, create the Y vector, and check the fraction of instances

---

In [162]:
Ycp = np.array((su_n.alchemy_category == 'recreation')).astype('int')
print Ycp[0:20]
print np.mean(Ycp)

[0 1 0 0 0 0 0 0 0 1 1 1 0 1 0 0 1 0 0 0]
0.243704144359


### 7.2 Use patsy to create an X matrix of the numeric predictors and all two-way interactions between them

Ex:

```python
import patsy

formula_interactions = '~ (var1 + var2 + var3)**2 -1'
X_interactions = patsy.dmatrix(formula_interactions, data=data
```

Get the column names from the `design_info` property of the patsy X matrix.

---

In [125]:
predictor_cols

['alchemy_category_score',
 'avglinksize',
 'commonlinkratio_1',
 'commonlinkratio_2',
 'commonlinkratio_3',
 'commonlinkratio_4',
 'compression_ratio',
 'frameTagRatio',
 'hasDomainLink',
 'html_ratio',
 'is_news',
 'lengthyLinkDomain',
 'linkwordscore',
 'news_front_page',
 'non_markup_alphanum_characters',
 'numberOfLinks',
 'numwords_in_url',
 'parametrizedLinkRatio',
 'spelling_errors_ratio']

In [164]:
su_n['is_recreation'] = Ycp

In [165]:
formula_inter = 'is_recreation ~ (' + ' + '.join(predictor_cols)+ ')**2'

Yinter, Xinter = patsy.dmatrices(formula_inter, data=su_n, return_type='dataframe')

print Yinter.shape, Xinter.shape

(5043, 1) (5043, 191)


In [166]:
Xinter = (Xinter - Xinter.mean()) / Xinter.std()

In [167]:
Xinter.fillna(value=0, inplace=True)
Xinter.replace(np.inf, 0, inplace=True)

### 7.4 Gridsearch a logistic regression to predict accuracy on your new target from the interaction predictors

Include Ridge and Lasso.

---

In [168]:
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split


Yin = np.ravel(Yinter.values)
Xin = Xinter.values

print Yin.shape, Xin.shape

lr = LogisticRegression()

lr_params = {
    'solver':['liblinear'],
    'penalty':['l1','l2'],
    'C':np.logspace(-5,1,20)
}

lr_gs = GridSearchCV(lr, lr_params, cv=5, verbose=1)

lr_gs.fit(Xin, Yin)

print lr_gs.best_params_
print lr_gs.best_score_


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    1.9s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:  1.7min


(5043,) (5043, 191)
Fitting 5 folds for each of 40 candidates, totalling 200 fits
{'penalty': 'l1', 'C': 0.01438449888287663, 'solver': 'liblinear'}
0.762641284949


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  1.8min finished


### 7.5 Build a logistic regression with the optimal parameters, and look at the coefficients

---

In [170]:
Yinter, Xinter = patsy.dmatrices(formula_inter, data=su_n, return_type='dataframe')


lr_inter = lr_gs.best_estimator_

coefs = pd.DataFrame({'variable':Xinter.columns,
                      'coefs':lr_inter.coef_[0],
                      'abs_coef':np.abs(lr_inter.coef_[0])})

coefs.sort_values(['abs_coef'], inplace=True, ascending=False)

print coefs[0:25]

print np.sum(coefs.coefs != 0)

print lr_inter.score(Xin, Yin)
print 1-np.mean(Yin)

     abs_coef     coefs                                      variable
1    0.369087 -0.369087                        alchemy_category_score
32   0.183081 -0.183081        alchemy_category_score:news_front_page
29   0.080794  0.080794                alchemy_category_score:is_news
10   0.056663 -0.056663                                    html_ratio
159  0.046110 -0.046110                         is_news:numberOfLinks
90   0.032002 -0.032002                  commonlinkratio_3:html_ratio
8    0.023976 -0.023976                                 frameTagRatio
172  0.020218  0.020218                   linkwordscore:numberOfLinks
173  0.018948  0.018948                 linkwordscore:numwords_in_url
82   0.013827  0.013827               commonlinkratio_2:numberOfLinks
101  0.011911 -0.011911               commonlinkratio_4:frameTagRatio
171  0.011130  0.011130  linkwordscore:non_markup_alphanum_characters
12   0.007870 -0.007870                             lengthyLinkDomain
106  0.007236 -0.007

### 7.6 Gridsearch parameters for a logistic regression with the same target and predictors, but score based on precision rather than accuracy

Look at the documentation.

---

## [BONUS] 8. Build models predicting from words

This is a bit of the NLP we covered in the pipeline lecture!

---

### 8.1 Choose 'body' or 'title' from the boilerplate to be the basis of your word predictors

You will need to parse the json from the boilerplate field.

---

In [ ]:
import json

### 8.2 Use CountVectorizer to create your predictor matrix from the string column

It is up to you what range of ngrams and features, and whether or not you want the columns binary or counts.

---

### 8.3 Gridsearch a logistic regression predicting accuracy of your chosen target category from word predictor matrix

---

### 8.4 Do the same as above, but score the gridsearch based on precision rather than accuracy

---

### 8.5 Build a logistic regression with optimal precision categories

Print out the top 20 or 25 word features as ranked by their coefficients.

---